In [ ]:
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img #import image need this
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense,Input
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.applications import VGG16
from keras.applications import VGG19
from keras.applications import ResNet50
from keras.applications import MobileNetV2
from keras.callbacks import ModelCheckpoint
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras import optimizers
import tensorflow as  tf
import keras.backend as K
import os
import random
import scipy
from keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix,roc_curve,roc_auc_score,auc
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.preprocessing.image import load_img, img_to_array

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
def history_plot(results): 
    # list all data in history
    print(results.history.keys())
    # summarize history for accuracy
    plt.figure(1)
    plt.plot(results.history['acc'])
    plt.plot(results.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.savefig('Accuracy',dpi=300)
    plt.show()

    # summarize history for loss
    plt.figure(2)
    plt.plot(results.history["loss"], label="loss")
    plt.plot(results.history["val_loss"], label="val_loss")
    plt.plot(np.argmin(results.history["val_loss"]), 
             np.min(results.history["val_loss"]), 
             marker="x", color="r", label="best model")
    plt.xlabel("Epochs")
    plt.ylabel("log_loss")
    plt.legend();
    plt.savefig('Loss',dpi=300)
    plt.show()

In [ ]:
img_width, img_height = 224 , 224
batch_size = 32
epochs = 300
seed = 42
class_mode = 'categorical'
date = '2019-06-17_VGG16'

In [ ]:
## Model way transfer learning
initial_model =VGG16(include_top=False,weights='imagenet')
input = Input(shape=(img_width,img_height,3),name = 'image_input')
output_vgg16_conv = initial_model(input)

x = Flatten()(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(3, activation='softmax', name='predictions')(x)
model = Model(input= input ,output = x)

In [ ]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# 設定checkpoint & callbacks
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
def callbacks(name):
    save_dir = "checkpoint"
    #     tl.files.exists_or_mkdir(save_dir)
    path=save_dir+'/model_check_'+name+'.h5'
    callbacks = [
        EarlyStopping(patience=55, verbose=1),
        ReduceLROnPlateau(factor=0.1, patience=10, min_lr=0.000001, verbose=1),
        ModelCheckpoint(path, verbose=1, save_best_only=True, save_weights_only=False)
#         ModelCheckpoint(path,monitor='val_acc', verbose=1,
#                         save_best_only=True, save_weights_only=False)
    ]
    return callbacks

In [ ]:
train_lung_op_path='./pneu_dataset/train/Lung Opacity/'
train_normal_path='./pneu_dataset/train/Normal/'
train_not_normal_path='./pneu_dataset/train/Not Normal/'
paths = [train_lung_op_path,train_normal_path,train_not_normal_path]
# list of each img
train_path = []
train_label = []
# dir 
# 0 = Lung Opacity
# 1 = Normal 
# 2 = Not Normal
for path in paths:
    for file in  os.listdir(path):
        if path == './pneu_dataset/train/Lung Opacity/':
            train_path.append(path+file)
            train_label.append(0)
        elif path == './pneu_dataset/train/Normal/':
            train_path.append(path+file)
            train_label.append(1)
        elif path == './pneu_dataset/train/Not Normal/':
            train_path.append(path+file)
            train_label.append(2)
        else :
            train_label.append(3) # accidient

In [ ]:
valid_lung_op_path='./pneu_dataset/validation/Lung Opacity/'
valid_normal_path='./pneu_dataset/validation/Normal/'
valid_not_normal_path='./pneu_dataset/validation/Not Normal/'
paths = [valid_lung_op_path,valid_normal_path,valid_not_normal_path]
# list of each img
valid_path = []
valid_label=[]
# dir 
for path in paths:
    for file in  os.listdir(path):
        if path == './pneu_dataset/validation/Lung Opacity/':
            valid_path.append(path+file)
            valid_label.append(0)
        elif path == './pneu_dataset/validation/Normal/':
            valid_path.append(path+file)
            valid_label.append(1)
        elif path == './pneu_dataset/validation/Not Normal/':
            valid_path.append(path+file)
            valid_label.append(2)
        else :
            valid_label.append(3)

In [ ]:
# path_img = train_path+valid_path
# label = train_label+valid_label

In [ ]:
# load image (train)
train_data = []
count=0
for path in train_path : 
    img=load_img(path,target_size = (img_width, img_height))
    img_array = img_to_array(img)
    train_data.append(img_array)
    

In [ ]:
# load image (validation)
valid_data = []
count=0
for path in valid_path : 
    img=load_img(path,target_size = (img_width, img_height))
    img_array = img_to_array(img)
    valid_data.append(img_array)


In [ ]:
# To array
train = np.asarray(train_data)
valid = np.asarray(valid_data)
t_label = to_categorical(np.asarray(train_label), 3) # separate to 3 class
v_label = to_categorical(np.asarray(valid_label), 3) # separate to 3 class

In [ ]:
# preprocessing 
train = train.astype('float32')/255.0
valid = valid.astype('float32')/255.0

In [ ]:
history=model.fit(x=train,y=t_label,validation_split=0.2,
                  epochs=epochs,batch_size=batch_size,verbose=1)

In [ ]:
history_plot(history)

In [ ]:
scores=model.evaluate(valid,v_label)
print()
print(scores)